<a href="https://colab.research.google.com/github/nhuyen183/LungCancerSupportSystem/blob/master/Copy_of_Copy_of_LungCancerDSS_preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://archive.apache.org/dist/spark/spark-3.1.1/spark-3.1.1-bin-hadoop3.2.tgz
!tar xf spark-3.1.1-bin-hadoop3.2.tgz
!pip install -q findspark
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.1-bin-hadoop3.2"
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:2 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:3 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:12 http://security.ubuntu.com/ubuntu bionic-security/main amd64 Packages [3,068 kB]
Get:13 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [83.3 kB]
Hit:14 http://ppa.l

# Create Spark entry points


In [2]:
from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession

In [3]:
sc = SparkContext(conf=SparkConf())
spark = SparkSession(sparkContext=sc)

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Step 1: Define the problem
What sorts of people were likely to have lung cancer?

# Step 2: Gather the data
The datasets can be found here: https://www.kaggle.com/datasets/aemreusta/brfss-2020-survey-data

# Step 3: Prepare data for consumption

In [6]:
!pip install kaggle

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [7]:
! mkdir ~/.kaggle

In [10]:
!cp /content/drive/MyDrive/kaggle.json ~/.kaggle/kaggle.json

In [11]:
! chmod 600 ~/.kaggle/kaggle.json

In [12]:
! kaggle datasets download aemreusta/brfss-2020-survey-data

 97% 47.0M/48.3M [00:03<00:00, 22.6MB/s]
100% 48.3M/48.3M [00:03<00:00, 14.8MB/s]


In [13]:
!ls

brfss-2020-survey-data.zip  spark-3.1.1-bin-hadoop3.2
drive			    spark-3.1.1-bin-hadoop3.2.tgz
sample_data


In [14]:
!unzip brfss-2020-survey-data.zip

Archive:  brfss-2020-survey-data.zip
  inflating: brfss2020.csv           


In [15]:
# load packages
import sys
print('Python version: {}'. format(sys.version))

import pandas as pd
print('Python version: {}'. format(pd.__version__))

import matplotlib
print('matplotlib version: {}'. format(matplotlib.__version__))

import numpy as np
print('numpy version: {}'. format(np.__version__))

import scipy as sp
print('scipy version: {}'. format(sp.__version__))

import IPython
from IPython import display # pretty printing of dataframe in Jupyter notebook
print('IPython version: {}'. format(IPython.__version__))

import pyspark
print('Apache Spark Pyspark version: {}'. format(pyspark.__version__)) # pyspark version

# misc libraries
import random
import time

# ignore warnings
import warnings
warnings.filterwarnings('ignore')
print('-'*25)

Python version: 3.7.15 (default, Oct 12 2022, 19:14:55) 
[GCC 7.5.0]
Python version: 1.3.5
matplotlib version: 3.2.2
numpy version: 1.21.6
scipy version: 1.7.3
IPython version: 7.9.0
Apache Spark Pyspark version: 3.1.1
-------------------------


In [17]:
from subprocess import check_output
print('-'*10, 'Files', '-'*10)
print(check_output(['ls', './']).decode('utf8'))

---------- Files ----------
brfss2020.csv
brfss-2020-survey-data.zip
drive
sample_data
spark-3.1.1-bin-hadoop3.2
spark-3.1.1-bin-hadoop3.2.tgz



# 
The Behavioral Risk Factor Surveillance System (BRFSS) is a collaborative project between all of the states in the United States and participating US territories and the Centers for Disease Control and Prevention (CDC).

BRFSS’s objective is to collect uniform state-specific data on health risk behaviors, chronic diseases and conditions, access to health care, and use of preventive health services related to the leading causes of death and disability in the United States. BRFSS conducts both landline and mobile phone-based surveys with individuals over the age of 18. General factors assessed by the BRFSS in 2020 included health status and healthy days, exercise, insufficient sleep, chronic health conditions, oral health, tobacco use, cancer screenings, and access to healthcare.

Survey data has been published annually by the CDC since 1984. You can find the original dataset as a ASCII format and past years data from here.

Survey data is shared by CDC in ASCII format. After converting the data to csv format, it was also published on Kaggle.

The aim of this project is to build a model with relatively high accuracy and AUC that could serve as an awareness-tool for those at high risk of developing heart disease.

The data contains information about 401958 unique survey participant. As a result of my research to select the ones related to coronary artery disease among a total of 279 different features, according to the information I got from the Texas Heart Institute website, there are 13 risk factors.

In [16]:
from pyspark.ml.classification import LinearSVC
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.classification import GBTClassifier
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.classification import NaiveBayes
from pyspark.ml.classification import MultilayerPerceptronClassifier
from pyspark.ml.classification import OneVsRest

In [18]:
data_raw = spark.read.csv('./brfss2020.csv', inferSchema=True, header=True)

# the test file provided is for validation of final model.
#data_val = spark.read.csv('./kaggle-titanic-test.csv', inferSchema=True, header=True)

# preview the data
# data type
print('-'*10, 'data types', '-'*10)
pd.DataFrame(data_raw.dtypes)

---------- data types ----------


,0,1
0,_STATE,double
1,FMONTH,double
2,IDATE,int
3,IMONTH,int
4,IDAY,int
...,...,...
274,_STOLDNA,double
275,_VIRCOLN,double
276,_SBONTIM,double
277,_CRCREC1,double


In [19]:
# data summary
print('-'*10, 'data summary', '-'*10)
data_raw.describe().toPandas()

---------- data summary ----------


,summary,_STATE,FMONTH,IDATE,IMONTH,IDAY,IYEAR,DISPCODE,SEQNO,_PSU,...,_RFPSA23,_CLNSCPY,_SGMSCPY,_SGMS10Y,_RFBLDS4,_STOLDNA,_VIRCOLN,_SBONTIM,_CRCREC1,_AIDTST4
0,count,401958,401958,401958,401958,401958,401958,401958,401958,401958,...,126604,177922,171486,171486,175853,176357,174274,194217,175698,367921
1,mean,30.010359291269236,5.959192253917076,6662701.046606859,6.510600610013982,15.008040641062001,2020.0301822578479,1117.7478741560064,2.0200045922818801E9,2.0200045922818801E9,...,2.3371615430792074,1.5802655096053326,2.851958760481905,2.8366630512111777,2.57898926944664,2.880906343383024,2.9384589783903508,2.5372959112745024,1.4172785119921685,2.040280386278576
2,stddev,15.773473494903223,3.5242238117738207,3462450.1811220297,3.4669000464972615,8.428138498040536,0.1710887546889567,38.207381620255546,3396.6841138105215,3396.6841138105215,...,2.2033784158573253,0.8728051431053163,0.44380827118133553,0.4879450778036646,0.6730066711378386,0.4513059766839771,0.32518874638973705,0.5119272764015419,0.7627272276885876,1.680522189372081
3,min,1.0,1.0,1012021,1,1,2020,1100.0,2020000001,2.020000001E9,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
4,max,72.0,12.0,12312020,12,31,2021,1200.0,2020015835,2.020015835E9,...,9.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,9.0


In [25]:
pd.DataFrame(data_raw.head())

,0
0,1.0
1,1.0
2,1042020.0
3,1.0
4,4.0
...,...
274,3.0
275,3.0
276,3.0
277,1.0
